<a href="https://colab.research.google.com/github/ruantos/python-para-data/blob/main/Learning_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the hard way

### Installing dependencies
1. install java
2. downloads spark tar
3. unzip it
4. pip install findspark


In [43]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

### Setting environment variables

In [44]:
from pyspark.sql import SparkSession

In [45]:
spark = SparkSession \
            .builder \
            .appName("Primeira Sessão") \
            .getOrCreate()

## Mounting your google drive

In [46]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Unziping files

In [47]:
# import zipfile

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/estabelecimentos.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/socios.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')


## Creating sparks dataframes

In [48]:
path = {
    "empresas": "/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas",
    "socios": "/content/drive/MyDrive/Colab Notebooks/Curso spark/socios"
    }

In [49]:
empresas = spark.read.csv(path["empresas"], sep=";", inferSchema=True)
# estabelecimentos = spark.read.csv(path["estabelecimentos"], sep=";", inferSchema=True)
socios = spark.read.csv(path["socios"], sep=";", inferSchema=True)

## Changing columns names

In [50]:
empresas.limit(5).show()

+----+--------------------+----+---+-------+---+----+
| _c0|                 _c1| _c2|_c3|    _c4|_c5| _c6|
+----+--------------------+----+---+-------+---+----+
| 306|FRANCAMAR REFRIGE...|2240| 49|   0,00|  1|NULL|
|1355|BRASILEIRO & OLIV...|2062| 49|   0,00|  5|NULL|
|4820|REGISTRO DE IMOVE...|3034| 32|   0,00|  5|NULL|
|5347|ROSELY APARECIDA ...|2135| 50|   0,00|  5|NULL|
|6846|BADU E FILHOS TEC...|2062| 49|4000,00|  1|NULL|
+----+--------------------+----+---+-------+---+----+



In [51]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [52]:
def rename_col(df, col_names):
    for index, name in enumerate(col_names):
        # enumerate retorna uma tupla contendo
        # o indice e cada valor no array

        # params são o nome da coluna e o novo nome
        df = df.withColumnRenamed(f"_c{index}", name)
    return df

In [53]:
empresas = rename_col(empresas, empresasColNames)
# estabelecimentos = rename_col(estabelecimentos, estabsColNames)
socios = rename_col(socios, sociosColNames)

In [54]:
empresas.limit(5).show()

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|        306|         FRANCAMAR REFRIGE...|             2240|                         49|                     0,00|               1|                       NULL|
|       1355|         BRASILEIRO & OLIV...|             2062|                         49|                     0,00|               5|                       NULL|
|       4820|         REGISTRO DE IMOVE...|             3034|                         32|                     0,00|               5|                       NULL|
|       5347|         ROSELY APARE

In [55]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



## Using replace and changing cols datatypes

In [56]:
from pyspark.sql.types import DecimalType, StringType, DoubleType
from pyspark.sql import functions as f

In [57]:
col = "capital_social_da_empresa"
empresas = empresas.withColumn(col, f.regexp_replace(col, ",", "."))

In [58]:
empresas = empresas.withColumn(col, empresas[col].cast(DoubleType()))

## Converting string to date

In [59]:
socios = socios.withColumn("data_de_entrada_sociedade",\
           f.to_date(socios.data_de_entrada_sociedade.cast(StringType()), "yyyyMMdd"))
# df.withColumn(col, f.to_date(df[col].cast(old_type()), "date_formatt") )

In [60]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



## SQL Queries

- Select
- Funções de data
- Alias
- Params do show

In [61]:
socios\
    .select("cnpj_basico" ,"identificador_de_socio",
            f.year("data_de_entrada_sociedade").alias("ano_entrada"),
            f.month("data_de_entrada_sociedade").alias("mes_entrada"))\
    .show(5, truncate=False)

+-----------+----------------------+-----------+-----------+
|cnpj_basico|identificador_de_socio|ano_entrada|mes_entrada|
+-----------+----------------------+-----------+-----------+
|411        |2                     |1994       |7          |
|411        |2                     |1994       |7          |
|5813       |2                     |1994       |5          |
|5813       |2                     |1994       |5          |
|14798      |2                     |1994       |6          |
+-----------+----------------------+-----------+-----------+
only showing top 5 rows



In [62]:
data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
colNames = ['nome', 'idade']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

+------------------------------+-----+
|nome                          |idade|
+------------------------------+-----+
|GISELLE PAULA GUIMARAES CASTRO|15   |
|ELAINE GARCIA DE OLIVEIRA     |22   |
|JOAO CARLOS ABNER DE LOURDES  |43   |
|MARTA ZELI FERREIRA           |24   |
|LAUDENETE WIGGERS ROEDER      |51   |
+------------------------------+-----+



## Query com substring
- substring
- concat

In [63]:
df.select(
    f.concat_ws(
        ", ",
        f.substring_index("nome", " ", 1),
        f.substring_index("nome", " ", -1)
        ).alias("ident"),
    "idade"
    )\
    .show(truncate=False)

+-----------------+-----+
|ident            |idade|
+-----------------+-----+
|GISELLE, CASTRO  |15   |
|ELAINE, OLIVEIRA |22   |
|JOAO, LOURDES    |43   |
|MARTA, FERREIRA  |24   |
|LAUDENETE, ROEDER|51   |
+-----------------+-----+



## Counting Nulls and NaN

In [64]:
socios\
    .select([f.count( f.when(f.isnull(c), 1)).alias(f"Qtd_nulls_em_{c}") for c in socios.columns])\
    .show()

+------------------------+-----------------------------------+------------------------------------------+---------------------------------+----------------------------------+--------------------------------------+-----------------+--------------------------------+----------------------------------+------------------------------------------------+-------------------------+
|Qtd_nulls_em_cnpj_basico|Qtd_nulls_em_identificador_de_socio|Qtd_nulls_em_nome_do_socio_ou_razao_social|Qtd_nulls_em_cnpj_ou_cpf_do_socio|Qtd_nulls_em_qualificacao_do_socio|Qtd_nulls_em_data_de_entrada_sociedade|Qtd_nulls_em_pais|Qtd_nulls_em_representante_legal|Qtd_nulls_em_nome_do_representante|Qtd_nulls_em_qualificacao_do_representante_legal|Qtd_nulls_em_faixa_etaria|
+------------------------+-----------------------------------+------------------------------------------+---------------------------------+----------------------------------+--------------------------------------+-----------------+-------------------

## Replacing NaN

In [65]:
# Se passar valor numérico, ele só preenche nas colunas numéricas
# O equivalente para outros types também se aplica

#Float e NaN, o resto é Null
socios.na.fill("-").show()
# socios.na.fill(0).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                   22|               1994-07-25|NULL|        ***000000**|                    -|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      

## Order By

In [66]:
socios\
    .select("nome_do_socio_ou_razao_social", "faixa_etaria",
            f.year("data_de_entrada_sociedade").alias("ano_entrada"))\
    .orderBy(["ano_entrada", "faixa_etaria"], ascending=[False, False])\
    .show(5, truncate=False)

+-------------------------------+------------+-----------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_entrada|
+-------------------------------+------------+-----------+
|EDGAR FRANCISCO DA SILVA       |9           |2021       |
|MARIA RAIMUNDA DOS SANTOS LANZA|9           |2021       |
|WILLIAM WHITING BEACH VEALE    |9           |2021       |
|MARIA JOSE DOMINGUES BONATO    |9           |2021       |
|ANTONIO TAVARES DE ANDRADE     |9           |2021       |
+-------------------------------+------------+-----------+
only showing top 5 rows



## Where clause

In [67]:
socios\
    .select(
        "nome_do_socio_ou_razao_social", "faixa_etaria",
        f.year("data_de_entrada_sociedade").alias("ano_entrada"))\
    .where("ano_entrada = 2021")\
    .limit(5)\
    .show()

+-----------------------------+------------+-----------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_entrada|
+-----------------------------+------------+-----------+
|              ROBERTA BENELLI|           4|       2021|
|         JOSE ALCEU DO ROS...|           5|       2021|
|          JOSE HUMBERTO PAIVA|           6|       2021|
|              MARCELO MOCELIN|           5|       2021|
|         EDUARDO DE ANDRAD...|           5|       2021|
+-----------------------------+------------+-----------+



## Where onde string 'começa com"

In [68]:
socios\
    .where(socios.nome_do_socio_ou_razao_social.startswith("JUAN"))\
    .show(5, truncate=False)

+-----------+----------------------+------------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social       |cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+------------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|7209603    |2                     |JUAN CARLOS MUNIZ RIVAS             |***260902**         |22                   |2005-02-10               |NULL|***000000**        |NULL                 |0                                  |6           |
|10207150   |2                     |JUAN CAR

## SQL LIKE

In [69]:
socios\
    .where(socios.nome_do_socio_ou_razao_social.like("%OLIVEIRA%"))\
    .show(5, truncate=False)

+-----------+----------------------+----------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social     |cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+----------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|323498     |2                     |ANA MARIA DE OLIVEIRA             |***786738**         |49                   |1994-11-24               |NULL|***000000**        |NULL                 |0                                  |6           |
|323498     |2                     |SONIA DE OLIVEIR

In [70]:
socios.show(5)

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                   22|               1994-07-25|NULL|        ***000000**|                 NULL|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      

## GroupBy and Aggregation functions

In [71]:
socios\
    .select(f.year("data_de_entrada_sociedade").alias("ano_entrada"))\
    .where("ano_entrada >= 2010")\
    .groupBy("ano_entrada")\
    .count()\
    .orderBy("ano_entrada", ascending=True)\
    .show(5)

+-----------+-----+
|ano_entrada|count|
+-----------+-----+
|       2010|79337|
|       2011|83906|
|       2012|80101|
|       2013|83919|
|       2014|80590|
+-----------+-----+
only showing top 5 rows



## Multiple aggregation functions

In [72]:
empresas\
    .select("cnpj_basico", "porte_da_empresa", "capital_social_da_empresa")\
    .groupBy("porte_da_empresa")\
    .agg(
        f.avg("capital_social_da_empresa").alias("capital_social_medio"),
        f.count("cnpj_basico").alias("Qtd_empresas_por_porte")
    )\
    .orderBy("porte_da_empresa", ascending=True)\
    .show(10)

+----------------+--------------------+----------------------+
|porte_da_empresa|capital_social_medio|Qtd_empresas_por_porte|
+----------------+--------------------+----------------------+
|            NULL|    8.35421888053467|                  5985|
|               1|  339994.53313506936|               3129043|
|               3|  2601001.7677092673|                115151|
|               5|   708660.4208249798|               1335500|
+----------------+--------------------+----------------------+



## Describe on Spark

In [73]:
empresas\
    .select("capital_social_da_empresa")\
    .summary("mean", "max")\
    .show()

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|   mean|        503694.5478542675|
|    max|         3.22014670262E11|
+-------+-------------------------+



In [74]:
socios\
    .select("faixa_etaria")\
    .summary("mean")\
    .show()

+-------+-----------------+
|summary|     faixa_etaria|
+-------+-----------------+
|   mean|5.396815918453111|
+-------+-----------------+



## When and Otherwise

In [75]:
data = [
    ('CARLOS', 'MATEMÁTICA', 7),
    ('IVO', 'MATEMÁTICA', 9),
    ('MÁRCIA', 'MATEMÁTICA', 8),
    ('LEILA', 'MATEMÁTICA', 9),
    ('BRENO', 'MATEMÁTICA', 7),
    ('LETÍCIA', 'MATEMÁTICA', 8),
    ('CARLOS', 'FÍSICA', 2),
    ('IVO', 'FÍSICA', 8),
    ('MÁRCIA', 'FÍSICA', 10),
    ('LEILA', 'FÍSICA', 9),
    ('BRENO', 'FÍSICA', 1),
    ('LETÍCIA', 'FÍSICA', 6),
    ('CARLOS', 'QUÍMICA', 10),
    ('IVO', 'QUÍMICA', 8),
    ('MÁRCIA', 'QUÍMICA', 1),
    ('LEILA', 'QUÍMICA', 10),
    ('BRENO', 'QUÍMICA', 7),
    ('LETÍCIA', 'QUÍMICA', 9)
]
colNames = ['nome', 'materia', 'nota']
df = spark.createDataFrame(data, colNames)

In [76]:
df = df.withColumn("status",
                   f.when(df.nota >= 7, "APROVADO").otherwise("REPROVADO"))

## Queries sql

In [77]:
empresas.createOrReplaceTempView("empresasView")

spark.sql("""

    SELECT *
    FROM empresasView
    LIMIT 5;
""").show()

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|        306|         FRANCAMAR REFRIGE...|             2240|                         49|                      0.0|               1|                       NULL|
|       1355|         BRASILEIRO & OLIV...|             2062|                         49|                      0.0|               5|                       NULL|
|       4820|         REGISTRO DE IMOVE...|             3034|                         32|                      0.0|               5|                       NULL|
|       5347|         ROSELY APARE

In [80]:
## Saving spark df as CSV

In [81]:
socios.write.csv(
    path="/content/drive/MyDrive/Colab Notebooks/Curso spark/socios/csv",
    mode="overwrite",
    sep=";",
    header=True,

)

In [86]:
## Spark methods are more efficient with parquet

In [85]:
empresas.write.parquet(
    path="/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas/parquet",
    mode="overwrite",
)

In [93]:
empresas_csv = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas/csv",
    sep=";",
    header=True
    )

In [88]:
empresas_parquet = spark.read.parquet(
    "/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas/parquet"
    )

In [99]:
empresas_csv\
    .select("capital_social_da_empresa")\
    .summary()\
    .show()

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|  count|                  4585679|
|   mean|        503694.5478542675|
| stddev|     2.1118691490537405E8|
|    min|                      0.0|
|    25%|                      0.0|
|    50%|                   1000.0|
|    75%|                   7000.0|
|    max|                9999998.0|
+-------+-------------------------+



In [100]:
empresas_parquet\
    .select("capital_social_da_empresa")\
    .summary()\
    .show()

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|  count|                  4585679|
|   mean|        503694.5478542675|
| stddev|     2.1118691490537405E8|
|    min|                      0.0|
|    25%|                      0.0|
|    50%|                   1000.0|
|    75%|                   7000.0|
|    max|         3.22014670262E11|
+-------+-------------------------+



In [101]:
spark.stop()